<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-5/blob/main/ColoreadoTerritorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Coloreado de imagenes aereas del territorio Riojano

Para comenzar con el trabajo, vamos a descargar el dataset de las imagenes de territorio.

In [14]:
import urllib.request 

import os
from imutils import paths
import argparse

In [16]:
import sys
sys.argv=['']
del sys

In [17]:
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", type=str)
ap.add_argument("-a", "--anio", type=str)
ap.add_argument("-e", "--ext", type=str)
ap.add_argument("-v", "--version", type=str,default='00')
args = vars(ap.parse_args())

In [18]:
dataset = args["dataset"] # "ortofoto-2012"
anio = args["anio"] # 2012
ext = args["ext"]  #"c200u89x75c24n"
ver = args["version"]

In [20]:
os.mkdir("Territorio")

In [19]:
for i in range(162,198):
    for j in range(66,101):
        urllib.request.urlretrieve("https://www.iderioja.larioja.org/vfile.php?r=/sig/ficheros_web/ortofoto/"+dataset+"_"+ver+"/ov"+str(i)+"0"+str(j)+"_"+str(anio)+ver+"_"+ext+".jpg", dataset+"/"+ str(i)+"0"+str(j)+".jpg")

TypeError: ignored